## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv", index_col=0)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
City_ID,,,,,,,,,
0,Atuona,PF,-9.8000,-139.0333,77.20,78,18,20.87,light rain
1,Vila Franca Do Campo,PT,37.7167,-25.4333,73.96,62,26,8.43,scattered clouds
2,Cape Town,ZA,-33.9258,18.4232,63.88,62,1,5.99,clear sky
3,Laguna,US,38.4210,-121.4238,75.09,61,0,1.01,clear sky
4,Roma,US,43.2128,-75.4557,70.95,67,86,1.99,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   193
Country                191
Lat                    193
Lng                    193
Max Temp               193
Humidity               193
Cloudiness             193
Wind Speed             193
Current Description    193
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df= preferred_cities_df.dropna()
preferred_cities_df.count()

City                   191
Country                191
Lat                    191
Lng                    191
Max Temp               191
Humidity               191
Cloudiness             191
Wind Speed             191
Current Description    191
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
0,Atuona,PF,77.20,light rain,-9.8000,-139.0333,
3,Laguna,US,75.09,clear sky,38.4210,-121.4238,
12,Georgetown,MY,80.89,overcast clouds,5.4112,100.3354,
14,Cabo San Lucas,MX,84.51,overcast clouds,22.8909,-109.9124,
17,Maceio,BR,80.31,scattered clouds,-9.6658,-35.7353,
25,Goundi,TD,82.60,moderate rain,9.3627,17.3660,
26,Akyab,MM,80.96,overcast clouds,20.1500,92.9000,
28,Butaritari,KI,82.62,overcast clouds,3.0707,172.7902,
31,Akom,CM,80.26,light rain,2.6678,12.7109,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"    

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    hotels = requests.get(base_url, params=params).json()    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace("",np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [10]:
clean_hotel_df.head(50)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
0,Atuona,PF,77.20,light rain,-9.8000,-139.0333,Villa Enata
3,Laguna,US,75.09,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
12,Georgetown,MY,80.89,overcast clouds,5.4112,100.3354,Cititel Penang
14,Cabo San Lucas,MX,84.51,overcast clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
17,Maceio,BR,80.31,scattered clouds,-9.6658,-35.7353,Poutur Pousada
26,Akyab,MM,80.96,overcast clouds,20.1500,92.9000,Hotel Memory
28,Butaritari,KI,82.62,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
31,Akom,CM,80.26,light rain,2.6678,12.7109,Complexe Hôtelier Jeannette
34,Marsh Harbour,BS,84.02,overcast clouds,26.5412,-77.0636,Abaco Beach Resort


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))